In [1]:
%run "../../common/matrix.py"
%run "../../common/grid_search_svr.py"
%run "../../common/file_handler.py"
%run "../../common/svr_normalization.py"

In [2]:
from sklearn.model_selection import KFold

### Import matrix

In [3]:
imported_matrix = open_file_from_fullpath(ask_file_path(), 'r')
matrix_to_normalize = pd.read_perseus(imported_matrix)

In [4]:
json_file = open_file_from_fullpath(ask_file_path(), 'r')

##### Our imported matrix will be divided into two submatrices: one containing only the QCs and one containing the samples.

In [5]:
grid_name = 'SVR_'
export_suffix = ''
try:
    matrix_to_normalize = filter_matrix(matrix_to_normalize, get_value_from_json(json_file, 'relevant'), get_value_from_json(json_file, 'value'))
    grid_name += 'filtered_'
    export_suffix += '_filtered'
    if get_value_from_json(json_file, 'value') == 'True':
        grid_name += 'true_'
        export_suffix += '_true'
    else:
        grid_name += 'false_'
        export_suffix += '_false'
except KeyError:
    pass

In [6]:
qc_matrix = slice_submatrix(matrix_to_normalize, get_value_from_json(json_file, 'QC_slice'))
qc_dictionary, qc_matrix = rename_matrixheader(qc_matrix, get_value_from_json(json_file, 'QC_rename'))
qc_matrix = sort_by_injection_order(qc_matrix)

sample_matrix = slice_submatrix(matrix_to_normalize, get_value_from_json(json_file, 'sample_slice'))
sample_dictionary, sample_matrix = rename_matrixheader(sample_matrix, get_value_from_json(json_file, 'sample_rename'))

try:
    ext_matrix = slice_submatrix(matrix_to_normalize, get_value_from_json(json_file, 'EXT_slice'))
    ext_dictionary, ext_matrix = rename_matrixheader(ext_matrix, get_value_from_json(json_file, 'EXT_rename'))
    sample_matrix = pd.concat((sample_matrix, ext_matrix), axis=1)
    merge_dictionary = qc_dictionary | sample_dictionary | ext_dictionary
except KeyError:
    merge_dictionary = qc_dictionary |sample_dictionary
sample_matrix = sort_by_injection_order(sample_matrix)

qc_injection_order = get_injection_order_plugin(qc_matrix)
qc_metabolites = cut_metabolites(qc_matrix)
samples_injection_order = get_injection_order_plugin(sample_matrix)
samples_metabolites = cut_metabolites(sample_matrix)

#### Global parameters
SVR has different kernel methods, we've chosen rbf and poly 

In [7]:
params_c = [0.01, 0.1, 1, 10]
params_d = [2, 3, 4]
param_gamma = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
kernel_rbf = ['rbf']
kernel_poly = ['poly']
kfold = KFold(n_splits=4)
global_params_rbf = {'C': params_c, 'gamma': param_gamma, 'kernel': kernel_rbf}
global_params_poly = {'C': params_c, 'degree': params_d, 'kernel': kernel_poly}

###  We generate features and labels for QC and after we'll start the gridsearch. 

In [8]:
qc_grids_inj_ord, qc_grids_mets = generate_grid_search_features_labels_svr(qc_injection_order, qc_metabolites)

### RBF kernel

In [9]:
svr_grids_rbf = grid_search_svr(grid_name + 'RBF_' + get_file_name(imported_matrix), qc_grids_inj_ord, qc_grids_mets, global_params_rbf, cv = kfold, checkpoint_path=r'../../../pickle_models',
    checkpoint = True, single_estimator = False, checkpoint_step = 100, verbose = True, threads = -1)


Grid search completed!


### POLY kernel

In [10]:
svr_grid_poly = grid_search_svr(grid_name + 'POLY_' + get_file_name(imported_matrix), qc_grids_inj_ord, qc_grids_mets, global_params_poly, cv = kfold, checkpoint_path=r'../../../pickle_models',
    checkpoint = True, single_estimator = False, checkpoint_step = 100, verbose = True, threads = -1)

-> Processing 2301 of 2373 elements 	Checkpoint in 10 elements	Estimated time remaining 00:06:43
-> Processing 2302 of 2373 elements 	Checkpoint in 9 elements	Estimated time remaining 00:06:41
-> Processing 2303 of 2373 elements 	Checkpoint in 8 elements	Estimated time remaining 00:05:48
-> Processing 2304 of 2373 elements 	Checkpoint in 7 elements	Estimated time remaining 00:08:10
-> Processing 2305 of 2373 elements 	Checkpoint in 6 elements	Estimated time remaining 00:08:38
-> Processing 2306 of 2373 elements 	Checkpoint in 5 elements	Estimated time remaining 00:07:50
-> Processing 2307 of 2373 elements 	Checkpoint in 4 elements	Estimated time remaining 00:07:04
-> Processing 2308 of 2373 elements 	Checkpoint in 3 elements	Estimated time remaining 00:07:18
-> Processing 2309 of 2373 elements 	Checkpoint in 2 elements	Estimated time remaining 00:07:04
-> Processing 2310 of 2373 elements 	Checkpoint in 1 elements	Estimated time remaining 00:07:12
-> Processing 2311 of 2373 elements 	Ch

### Prediction on QC and samples
Following, execution of ratio values and peak intesity calculation. The intensity obtained with function ratio_to_intensity() is devoid of analytical drift

#### Run this cells below if you have chosen RBF kernel

In [11]:
qc_meta_normalized_rbf = prediction(svr_grids_rbf, qc_metabolites, qc_injection_order)
samples_meta_normalized_rbf = prediction(svr_grids_rbf, samples_metabolites, samples_injection_order)

In [12]:
ratio_values_sample_rbf = compute_ratio(samples_metabolites, samples_meta_normalized_rbf)
ratio_to_intensity_sample_rbf = ratio_to_intensity(ratio_values_sample_rbf, samples_metabolites)
ratio_values_qc_rbf = compute_ratio(qc_metabolites, qc_meta_normalized_rbf)
ratio_to_intensity_qc_rbf = ratio_to_intensity(ratio_values_qc_rbf, qc_metabolites)

#### Run this cells below if you have chosen POLY kernel

In [13]:
qc_meta_normalized_poly = prediction(svr_grid_poly, qc_metabolites, qc_injection_order)
samples_meta_normalized_poly = prediction(svr_grid_poly, samples_metabolites, samples_injection_order)

In [14]:
ratio_values_sample_poly = compute_ratio(samples_metabolites, samples_meta_normalized_poly)
ratio_to_intensity_sample_poly = ratio_to_intensity(ratio_values_sample_poly, samples_metabolites)
ratio_values_qc_poly = compute_ratio(qc_metabolites, qc_meta_normalized_poly)
ratio_to_intensity_qc_poly = ratio_to_intensity(ratio_values_qc_poly, qc_metabolites)

#### We'll obtain the matrix with normalized values (ratio)
##### RBF

In [15]:
matrix_sample_meta_ratio_rbf = paste_metabolites(sample_matrix, ratio_values_sample_rbf)
matrix_qc_meta_ratio_rbf = paste_metabolites(qc_matrix, ratio_values_qc_rbf)
merge_ratio_matrix_rbf = merge_submatrix_plugin(matrix_to_normalize, matrix_qc_meta_ratio_rbf, matrix_sample_meta_ratio_rbf, merge_dictionary)
merge_ratio_matrix_rbf

Column Name,X99_control_NB_Plasma_Quant_HILIC_Pos_137,X98_control_NB_Plasma_Quant_HILIC_Pos_057,X97_control_NB_Plasma_Quant_HILIC_Pos_054,X96_control_NB_Plasma_Quant_HILIC_Pos_039,X95_control_NB_Plasma_Quant_HILIC_Pos_069,X94_control_NB_Plasma_Quant_HILIC_Pos_086,X93_control_NB_Plasma_Quant_HILIC_Pos_135,X92_control_NB_Plasma_Quant_HILIC_Pos_064,X91_control_NB_Plasma_Quant_HILIC_Pos_022,X90_control_NB_Plasma_Quant_HILIC_Pos_034,...,Coefficient.of.variation.EXT,Coefficient.of.variation.control,Coefficient.of.variation.local,Coefficient.of.variation.post,Coefficient.of.variation.onset,Coefficient.of.variation.QC,Alignment.ID,Metabolite.name,Adduct.type,Post.curation.result
Group1,control,control,control,control,control,control,control,control,control,control,...,,,,,,,,,,
0,0.980235,0.980005,1.043557,1.036310,0.959525,1.036087,0.974394,1.048056,1.044584,0.992516,...,0.381733,0.598872,0.524052,0.592017,0.614022,0.352436,11,Unknown,[M+H-2H2O]+,found in higher mz's MsMs_3793;adduct linked t...
1,0.991879,1.035887,1.030778,1.002677,0.977604,1.006317,1.036319,1.004654,1.003997,1.036986,...,0.071236,0.435502,0.484476,0.226847,0.307424,0.150061,12,Unknown,[M+H-H2O]+,found in higher mz's MsMs_3778;found in higher...
2,1.113202,1.056362,1.078301,1.023556,1.141816,1.068430,1.131323,1.083085,1.014301,1.081734,...,0.442347,0.570537,0.589903,0.545421,0.600632,0.339553,14,Unknown,[M+H-H2O]+,adduct linked to 1239_[M+H]+;found in higher m...
3,0.967860,0.984687,0.956987,0.998273,0.956141,0.936593,0.977536,0.992602,1.005935,0.967267,...,0.114283,0.291590,0.259090,0.273496,0.389428,0.190791,32,Unknown,[M+2H]+,found in higher mz's MsMs_10382;adduct linked ...
4,1.046220,1.056423,1.057499,1.024258,1.001808,1.052580,1.027524,1.031919,0.997780,1.025622,...,0.454047,0.409125,0.375197,0.422308,0.416197,0.361926,152,Pyrrolidine;CE10;RWRDLPDLKQPQOW-UHFFFAOYSA-N,[M+H]+,adduct linked to 3468_[M+ACN+Na]+;adduct linke...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,0.997993,0.953307,0.991298,0.996095,1.004448,0.987855,1.051471,1.035216,0.979062,1.036360,...,0.356886,0.504725,0.427966,0.356015,0.370765,0.305047,41130,Unknown,[M+2H]2+,NaN
2369,0.864723,0.580252,0.648388,0.699531,0.878227,0.825807,0.816502,0.872777,0.872996,0.651281,...,NaN,NaN,0.581382,0.674037,0.552842,0.390473,41208,Unknown,[M+2H]2+,NaN
2370,0.994183,0.953237,1.010357,0.960889,1.018355,0.968030,1.062166,1.127898,0.968845,0.971289,...,0.427014,0.845708,0.685381,0.717962,1.098157,0.268731,41224,Unknown,[M+H]+,NaN


#### path to save matrix

In [16]:
merge_ratio_matrix_rbf.to_perseus(generate_new_file_path(imported_matrix, '_ratio_svr_rbf' + export_suffix))

#### Poly

In [17]:
matrix_sample_meta_ratio_poly = paste_metabolites(sample_matrix, ratio_values_sample_poly)
matrix_qc_meta_ratio_poly = paste_metabolites(qc_matrix, ratio_values_qc_poly)
merge_ratio_matrix_poly = merge_submatrix_plugin(matrix_to_normalize, matrix_qc_meta_ratio_poly, matrix_sample_meta_ratio_poly, merge_dictionary)
merge_ratio_matrix_poly

Column Name,X99_control_NB_Plasma_Quant_HILIC_Pos_137,X98_control_NB_Plasma_Quant_HILIC_Pos_057,X97_control_NB_Plasma_Quant_HILIC_Pos_054,X96_control_NB_Plasma_Quant_HILIC_Pos_039,X95_control_NB_Plasma_Quant_HILIC_Pos_069,X94_control_NB_Plasma_Quant_HILIC_Pos_086,X93_control_NB_Plasma_Quant_HILIC_Pos_135,X92_control_NB_Plasma_Quant_HILIC_Pos_064,X91_control_NB_Plasma_Quant_HILIC_Pos_022,X90_control_NB_Plasma_Quant_HILIC_Pos_034,...,Coefficient.of.variation.EXT,Coefficient.of.variation.control,Coefficient.of.variation.local,Coefficient.of.variation.post,Coefficient.of.variation.onset,Coefficient.of.variation.QC,Alignment.ID,Metabolite.name,Adduct.type,Post.curation.result
Group1,control,control,control,control,control,control,control,control,control,control,...,,,,,,,,,,
0,1.002912,0.995157,1.061493,1.053110,0.953914,1.036614,0.993548,1.046942,1.052527,1.009973,...,0.381733,0.598872,0.524052,0.592017,0.614022,0.352436,11,Unknown,[M+H-2H2O]+,found in higher mz's MsMs_3793;adduct linked t...
1,0.991830,1.024575,1.018673,1.006706,0.980850,1.010352,1.035604,1.003794,1.010105,1.042561,...,0.071236,0.435502,0.484476,0.226847,0.307424,0.150061,12,Unknown,[M+H-H2O]+,found in higher mz's MsMs_3778;found in higher...
2,1.116582,1.089687,1.111484,1.023996,1.135979,1.095359,1.139421,1.086422,1.026308,1.098068,...,0.442347,0.570537,0.589903,0.545421,0.600632,0.339553,14,Unknown,[M+H-H2O]+,adduct linked to 1239_[M+H]+;found in higher m...
3,0.972052,0.984797,0.958846,0.976954,0.953134,0.922463,0.979754,0.990300,0.997947,0.959227,...,0.114283,0.291590,0.259090,0.273496,0.389428,0.190791,32,Unknown,[M+2H]+,found in higher mz's MsMs_10382;adduct linked ...
4,1.078183,1.035603,1.045910,0.990478,0.991298,1.037740,1.056344,1.010729,0.973254,1.001012,...,0.454047,0.409125,0.375197,0.422308,0.416197,0.361926,152,Pyrrolidine;CE10;RWRDLPDLKQPQOW-UHFFFAOYSA-N,[M+H]+,adduct linked to 3468_[M+ACN+Na]+;adduct linke...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,0.982816,0.964698,1.001615,1.004056,1.007163,0.995106,1.036455,1.046708,0.995833,1.052843,...,0.356886,0.504725,0.427966,0.356015,0.370765,0.305047,41130,Unknown,[M+2H]2+,NaN
2369,0.869759,0.579611,0.648641,0.675422,0.835204,0.833730,0.815192,0.855062,0.848193,0.624162,...,NaN,NaN,0.581382,0.674037,0.552842,0.390473,41208,Unknown,[M+2H]2+,NaN
2370,0.993904,0.952017,1.008836,0.959907,1.017117,0.966699,1.061808,1.126475,0.967607,0.970050,...,0.427014,0.845708,0.685381,0.717962,1.098157,0.268731,41224,Unknown,[M+H]+,NaN


#### path to save matrix

In [18]:
merge_ratio_matrix_poly.to_perseus(generate_new_file_path(imported_matrix, '_ratio_svr_poly' + export_suffix))

#### We'll obtain the original matrix with normalized values (intesity normalized)

#### RBF

In [19]:
matrix_sample_metab_norm_rbf = paste_metabolites(sample_matrix, ratio_to_intensity_sample_rbf)
matrix_qc_metab_norm_rbf = paste_metabolites(qc_matrix, ratio_to_intensity_qc_rbf)
merge_intensity_matrix_rbf = merge_submatrix_plugin(matrix_to_normalize, matrix_qc_metab_norm_rbf, matrix_sample_metab_norm_rbf, merge_dictionary)
merge_intensity_matrix_rbf

Column Name,X99_control_NB_Plasma_Quant_HILIC_Pos_137,X98_control_NB_Plasma_Quant_HILIC_Pos_057,X97_control_NB_Plasma_Quant_HILIC_Pos_054,X96_control_NB_Plasma_Quant_HILIC_Pos_039,X95_control_NB_Plasma_Quant_HILIC_Pos_069,X94_control_NB_Plasma_Quant_HILIC_Pos_086,X93_control_NB_Plasma_Quant_HILIC_Pos_135,X92_control_NB_Plasma_Quant_HILIC_Pos_064,X91_control_NB_Plasma_Quant_HILIC_Pos_022,X90_control_NB_Plasma_Quant_HILIC_Pos_034,...,Coefficient.of.variation.EXT,Coefficient.of.variation.control,Coefficient.of.variation.local,Coefficient.of.variation.post,Coefficient.of.variation.onset,Coefficient.of.variation.QC,Alignment.ID,Metabolite.name,Adduct.type,Post.curation.result
Group1,control,control,control,control,control,control,control,control,control,control,...,,,,,,,,,,
0,24.329084,23.933143,27.179060,26.756823,22.480313,26.416313,23.951265,26.939334,26.940275,24.571515,...,0.381733,0.598872,0.524052,0.592017,0.614022,0.352436,11,Unknown,[M+H-2H2O]+,found in higher mz's MsMs_3793;adduct linked t...
1,26.583021,28.350779,28.041507,26.928531,25.642744,27.243217,28.988208,26.955325,27.034074,28.834004,...,0.071236,0.435502,0.484476,0.226847,0.307424,0.150061,12,Unknown,[M+H-H2O]+,found in higher mz's MsMs_3778;found in higher...
2,25.410723,23.255746,24.210229,21.161817,26.223589,23.697003,26.338184,23.781722,21.012707,23.980019,...,0.442347,0.570537,0.589903,0.545421,0.600632,0.339553,14,Unknown,[M+H-H2O]+,adduct linked to 1239_[M+H]+;found in higher m...
3,22.495489,23.455017,22.196054,23.595783,22.033855,20.866451,22.916009,23.770729,24.289512,22.448782,...,0.114283,0.291590,0.259090,0.273496,0.389428,0.190791,32,Unknown,[M+2H]+,found in higher mz's MsMs_10382;adduct linked ...
4,25.541066,26.076688,26.378378,24.244895,23.599472,25.789248,24.636757,24.819913,23.231069,24.545650,...,0.454047,0.409125,0.375197,0.422308,0.416197,0.361926,152,Pyrrolidine;CE10;RWRDLPDLKQPQOW-UHFFFAOYSA-N,[M+H]+,adduct linked to 3468_[M+ACN+Na]+;adduct linke...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,25.537924,23.298447,25.139067,25.259123,25.698181,25.088372,28.347983,27.492627,24.578078,27.539094,...,0.356886,0.504725,0.427966,0.356015,0.370765,0.305047,41130,Unknown,[M+2H]2+,NaN
2369,12.982453,6.116768,7.651420,8.603686,13.316334,12.439648,11.525711,13.560259,13.487885,7.403402,...,NaN,NaN,0.581382,0.674037,0.552842,0.390473,41208,Unknown,[M+2H]2+,NaN
2370,19.558274,17.979844,20.194762,18.274806,20.520871,18.538165,22.324561,25.172388,18.574056,18.667875,...,0.427014,0.845708,0.685381,0.717962,1.098157,0.268731,41224,Unknown,[M+H]+,NaN


##### Path to save matrix

In [20]:
merge_intensity_matrix_rbf.to_perseus(generate_new_file_path(imported_matrix, '_intensity_svr_rbf' + export_suffix))

#### POLY

In [21]:
matrix_sample_metab_norm_poly = paste_metabolites(sample_matrix, ratio_to_intensity_sample_poly)
matrix_qc_metab_norm_poly = paste_metabolites(qc_matrix, ratio_to_intensity_qc_poly)
merge_intensity_matrix_poly = merge_submatrix_plugin(matrix_to_normalize,matrix_qc_metab_norm_poly, matrix_sample_metab_norm_poly, merge_dictionary)
merge_intensity_matrix_poly

Column Name,X99_control_NB_Plasma_Quant_HILIC_Pos_137,X98_control_NB_Plasma_Quant_HILIC_Pos_057,X97_control_NB_Plasma_Quant_HILIC_Pos_054,X96_control_NB_Plasma_Quant_HILIC_Pos_039,X95_control_NB_Plasma_Quant_HILIC_Pos_069,X94_control_NB_Plasma_Quant_HILIC_Pos_086,X93_control_NB_Plasma_Quant_HILIC_Pos_135,X92_control_NB_Plasma_Quant_HILIC_Pos_064,X91_control_NB_Plasma_Quant_HILIC_Pos_022,X90_control_NB_Plasma_Quant_HILIC_Pos_034,...,Coefficient.of.variation.EXT,Coefficient.of.variation.control,Coefficient.of.variation.local,Coefficient.of.variation.post,Coefficient.of.variation.onset,Coefficient.of.variation.QC,Alignment.ID,Metabolite.name,Adduct.type,Post.curation.result
Group1,control,control,control,control,control,control,control,control,control,control,...,,,,,,,,,,
0,24.891906,24.303160,27.646203,27.190566,22.348868,26.429767,24.422086,26.910711,27.145132,25.003687,...,0.381733,0.598872,0.524052,0.592017,0.614022,0.352436,11,Unknown,[M+H-2H2O]+,found in higher mz's MsMs_3793;adduct linked t...
1,26.581702,28.041183,27.712191,27.036741,25.727906,27.352449,28.968222,26.932244,27.198519,28.989010,...,0.071236,0.435502,0.484476,0.226847,0.307424,0.150061,12,Unknown,[M+H-H2O]+,found in higher mz's MsMs_3778;found in higher...
2,25.487880,23.989396,24.955276,21.170928,26.089530,24.294270,26.526722,23.854979,21.261430,24.342109,...,0.442347,0.570537,0.589903,0.545421,0.600632,0.339553,14,Unknown,[M+H-H2O]+,adduct linked to 1239_[M+H]+;found in higher m...
3,22.592910,23.457639,22.239168,23.091856,21.964555,20.551636,22.968019,23.715607,24.096639,22.262205,...,0.114283,0.291590,0.259090,0.273496,0.389428,0.190791,32,Unknown,[M+2H]+,found in higher mz's MsMs_10382;adduct linked ...
4,26.321371,25.562770,26.089291,23.445304,23.351894,25.425647,25.327788,24.310239,22.660051,23.956666,...,0.454047,0.409125,0.375197,0.422308,0.416197,0.361926,152,Pyrrolidine;CE10;RWRDLPDLKQPQOW-UHFFFAOYSA-N,[M+H]+,adduct linked to 3468_[M+ACN+Na]+;adduct linke...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368,25.149542,23.576850,25.400697,25.460984,25.767635,25.272536,27.943159,27.797837,24.999111,27.977081,...,0.356886,0.504725,0.427966,0.356015,0.370765,0.305047,41130,Unknown,[M+2H]2+,NaN
2369,13.058070,6.110011,7.654407,8.307167,12.664000,12.558987,11.507230,13.285017,13.104676,7.095133,...,NaN,NaN,0.581382,0.674037,0.552842,0.390473,41208,Unknown,[M+2H]2+,NaN
2370,19.552787,17.956831,20.164354,18.256130,20.495939,18.512677,22.317023,25.140612,18.550309,18.644067,...,0.427014,0.845708,0.685381,0.717962,1.098157,0.268731,41224,Unknown,[M+H]+,NaN


#### Path to save matrix

In [22]:
merge_intensity_matrix_poly.to_perseus(generate_new_file_path(imported_matrix, '_intensity_svr_poly' + export_suffix))

In [23]:
imported_matrix.close()
json_file.close()